# How to perform regression models

This is a detailed, step-by-step guide on how to perform the following regression models in Python.

* Multiple linear regression (forward method), 
* Regression tree, 
* Random forest, 
* Gradient boosting tree
* Support vector regression.

Terminology used in this guide:



## 1. Install and import the necesary packages and libraries

I already have the most recent versions of **pandas, numpy, seaborn and matplotlib** installed, but you can install them using pip (see pypi.org) or conda install in Anaconda prompt (see anaconda.org). If you get the ImportError: cannot import name 'html5lib' from 'pip._vendor', you can install html5lib in Anaconda prompt (conda install -c anaconda html5lib).

Currently installed versions: 
<br>Pandas 1.4.4
<br>numpy 1.21.5
<br>seaborn 0.12.2
<br>matplotlib 3.5.1
<br>scikit learn 1.1.1

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from tabulate import tabulate
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV

## 2. Read csv file into Pandas dataframe

In [ ]:
# Read the csv file into the pandas dataframe
df2 = pd.read_csv('filename.csv')

# If the rows are truncated so we can't see the full list, you can correct that with:
pd.set_option('display.max_rows', None)

# Let's display max columns too 
pd.set_option('display.max_columns', None)

# To see if the file loaded correctly
df2.head()

## 3. Feature selection

We will use individual correlations to select features instead of sequential feature selection (see vcoetzee/compare for why I prefer individual correlations).

In [ ]:
# Get the full list of features in the dataset
df2.columns

In [ ]:
# Select the variables you want to use
df3=df2[['variable1', 'variable2','variable3']]

In [ ]:
# Identify the variables most highly correlated with your dependent variable (in this case variable1)
df3.corrwith(df3['variable1']).sort_values(ascending=False)

In [ ]:
# Visualise the relationships
# Scatterplot between two continious variables, with a third variable illustrated by hue (optional)
print(sns.scatterplot(data=df3, x='variable2', y='variable1', hue='variable3'))

# Boxplot between a categorical (or ordinal) variable and your continious dependent variable
print(sns.boxplot(data=mm, x='variable4', y='variable1'))

## 4. Forward linear regression

In [ ]:
# Define testing and traing set
# (.values creates a numpy array)
x = df2[['variable2', 'variable3', 'variable4']].values
y = df2[['variable1']].values 

# Splitting the dataset into training and test set (80/20 split)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

In [ ]:
# Fit the model - according to correlations
# Step 1
x_train1 = x_train[:, [2]] # your most highly correlated x variable
x_test1 = x_test[:, [2]]
lr1 = LinearRegression()
lr1.fit(x_train1, y_train)

# Evaluate the model
y_pred_lr1=lr1.predict(x_test1)

# Evaluate the predictions
lr_R21 = r2_score(y_test, y_pred_lr1)
lr_MSE1 = mean_squared_error(y_test, y_pred_lr1)
print('Step1 R2:', lr_R21)
print('Step1 MSE:', lr_MSE1)
print()
  
#_________________________________
# Step 2
x_train2 = x_train[:, [2,3]]  # your two most highly correlated x variables
x_test2 = x_test[:, [2,3]]
lr2 = LinearRegression()
lr2.fit(x_train2, y_train)

# Evaluate the model
y_pred_lr2 = lr2.predict(x_test2)

# Evaluate the predictions
lr_R22 = r2_score(y_test, y_pred_lr2)
lr_MSE2 = mean_squared_error(y_test, y_pred_lr2)
print('Step2 R2:', lr_R22)
print('Step2 MSE:', lr_MSE2)
print()
  
#____________________________________
# Step 3 (etc)

# Identify the best model (i.e. the one with the highest R2 and lowest MSE)
# I keep adding variables/ steps until R2 have peaked (i.e. the last model should have a lower R2 than the one before)

In [ ]:
# Check for multicolinearity in your best model 

# Convert numpy to df
vif_df = pd.DataFrame(x_train2, columns = ['variable2', 'variable3'])

# Convert datatypes to float
vif_df[['variable2', 'variable3']] = vif_df[['variable2', 'variable3']].astype('float')

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = vif_df.columns

  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(vif_df.values, i)
                          for i in range(len(vif_df.columns))]

  
print(vif_data)

# Generally, a VIF above 5 indicates a high multicollinearity and a VIF above 10 needs to be corrected

## 5. Regression tree

In [ ]:
# Create the regressor
rt = DecisionTreeRegressor(random_state=1)

# Train the regressor 
rt.fit(x_train, y_train)

# Predicting the test data
y_pred_rt = rt.predict(x_test)

# Evaluate the predictions
# Calculate R2 score (higher R2=better performance) and MSE scores (lower MSE = better performance)
rt_R2 = r2_score(y_test, y_pred_rt)
rt_MSE = mean_squared_error(y_test, y_pred_rt)
print('Regression tree R2:', rt_R2)
print('Regression tree MSE:', rt_MSE)
print()

In [ ]:
# Identify the ideal max_depth value to use for pruning the tree
# (Regression trees tend to overfit, so it is important to prune the tree)
depth=[]
for i in range(1,5):
    regressor = DecisionTreeRegressor(random_state=1, criterion='squared_error', max_depth=i)
    model = regressor.fit(x_train, y_train)
    # Perform 5-fold cross validation 
    model_scores = cross_val_score(model, x, y, cv = 3, scoring='r2')
    print("mean R2 cross validation score: {}".format(np.mean(model_scores)))

    # Predict the response for test dataset
    y_pred = model.predict(x_test)
    depth.append(mean_squared_error(y_test, y_pred))
print('MSE:', depth)

In [ ]:
# Create the regressor for the optimal max_depth value identified above
rt3 = DecisionTreeRegressor(max_depth=3, random_state=1)

# Train the regressor 
rt3.fit(x_train, y_train)

# Predicting the test data
y_pred_rt3 = rt3.predict(x_test)

# Evaluate the predictions
# Calculate R2 score (higher R2=better performance) and MSE scores (lower MSE = better performance)
rt3_R2 = r2_score(y_test, y_pred_rt3)
rt3_MSE = mean_squared_error(y_test, y_pred_rt3)
print('Regression tree R2:', rt3_R2)
print('Regression tree MSE:', rt3_MSE)
print()

# plot tree
# set plot size (denoted in inches)
plt.figure(figsize=(15,12))
tree.plot_tree(rt3, feature_names=['variable2', 'variable3', 'variable4'])

## 4. Random Forest

In [ ]:
# Flatten y (otherwise you get the error: 
   # DataConversionWarning: A column-vector y was passed when a 1d array was expected. 
   # Please change the shape of y to (n_samples,), for example using ravel().)
y_train1_1 = np.ravel(y_train, order = 'C')
print('y_train shape:', y_train1_1.shape)
y_test1_1 = np.ravel(y_test, order = 'C')
print('y_test shape:', y_test1_1.shape)

# Create the regressor
# You can change the number of estimators
rf = RandomForestRegressor(n_estimators = 1000, random_state=1)

# Train the regressor
rf.fit(x_train, y_train1_1) 

# Predicting the test data
y_pred_rf = rf.predict(x_test)

# Evaluate the predictions
# Calculate R2 score (higher R2=better performance) and MSE scores (lower MSE = better performance)
rf_R2 = r2_score(y_test1_1, y_pred_rf)
rf_MSE = mean_squared_error(y_test, y_pred_rf)
print('Random Forest R2:', rf_R2)
print('Random Forest MSE:', rf_MSE)

## 5. Gradient boosting tree

In [ ]:
# Create the regressor
gb = GradientBoostingRegressor(random_state=1)

# Train the regressor
gb.fit(x_train, y_train1_1) 

# Predicting the test data
y_pred_gb = gb.predict(x_test)

# Evaluate the predictions
# Calculate R2 score (higher R2=better pegbormance) and MSE scores (lower MSE = better pegbormance)
gb_R2 = r2_score(y_test1_1, y_pred_gb)
gb_MSE = mean_squared_error(y_test, y_pred_rf)
print('Gradient Boost R2:', gb_R2)
print('Gradient Boost MSE:', gb_MSE)

## 6. Support Vector Regression

In [ ]:
# Create the regressor
svr = svm.SVR()

# Train the regressor
svr.fit(x_train, y_train1_1) 

# Predicting the test data
y_pred_svr = svr.predict(x_test)

# Evaluate the predictions
# Calculate R2 score (higher R2=better pesvrormance) and MSE scores (lower MSE = better pesvrormance)
svr_R2 = r2_score(y_test1_1, y_pred_svr)
svr_MSE = mean_squared_error(y_test, y_pred_svr)
print('Support vector R2:', svr_R2)
print('Support vector MSE:', svr_MSE)

In [ ]:
# Print a model comparison table
print('Model comparison')
table = [['Description', 'R2', 'MSE'], ['Linear regr', lr_R24, lr_MSE4], 
         ['Regr Tree', rt3_R2, rt3_MSE], ['Random Forest', rf_R2, rf_MSE], 
         ['Gradient Boost', gb_R2, gb_MSE], ['Support vector', svr_R2, svr_MSE]]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [ ]:
# You can perform hyperparameter tuning for the random forest model
   # if you want to improve it somewhat (optional)

# Create a randomised hyperparameter grid
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters

# First create the base model to tune
rf2 = RandomForestRegressor()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf2_random = RandomizedSearchCV(estimator = rf2, param_distributions = random_grid, 
                                n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf2_random.fit(x_train, y_train1_1) 

In [ ]:
# Get parameters for the best model
rf2_random.best_params_

In [ ]:
# Create the regressor
# (Use the best parameters obtained above)
rf2 = RandomForestRegressor(n_estimators=1000,min_samples_split=5,min_samples_leaf=4, 
                            max_features='sqrt', max_depth=30, bootstrap=False)

# Train the regressor
rf2.fit(x_train, y_train1_1) 

# Predicting the test data
y_pred_rf2 = rf2.predict(x_test)

# Evaluate the predictions
rf2_R2 = r2_score(y_test1_1, y_pred_rf2)
rf2_MSE = mean_squared_error(y_test, y_pred_rf2)

print('Random Forest R2:', rf2_R2)
print('Random Forest MSE:', rf2_MSE)

In [ ]:
# Print the model comparison table with the latest model included
print('Model comparison')
table2 = [['Description', 'R2', 'MSE'], ['Linear regr', lr_R24, lr_MSE4], 
          ['Regr Tree', rt3_R2, rt3_MSE], ['Random Forest1', rf_R2, rf_MSE], 
          ['Random Forest2', rf2_R2, rf2_MSE], ['Gradient Boost', gb_R2, gb_MSE], 
          ['Support vector', svr_R2, svr_MSE]]
print(tabulate(table2, headers='firstrow', tablefmt='fancy_grid'))